In [94]:
%%writefile mapper.py
import sys
import re
reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode
stop_words = ['a','about','above','across','after','afterwards','again','against','all','almost','alone','along','already','also','although','always','am','among','amongst','amoungst','amount','an','and','another','any','anyhow','anyone','anything','anyway','anywhere','are','around','as','at','back','be','became','because','become','becomes','becoming','been','before','beforehand','behind','being','below','beside','besides','between','beyond','bill','both','bottom','but','by','call','can','cannot','cant','co','computer','con','could','couldnt','cry','de','describe','detail','do','done','down','due','during','each','eg','eight','either','eleven','else','elsewhere','empty','enough','etc','even','ever','every','everyone','everything','everywhere','except','few','fifteen','fify','fill','find','fire','first','five','for','former','formerly','forty','found','four','from','front','full','further','get','give','go','had','has','hasnt','have','he','hence','her','here','hereafter','hereby','herein','hereupon','hers','herse\"','him','himse\"','his','how','however','hundred','i','ie','if','in','inc','indeed','interest','into','is','it','its','itse\"','keep','last','latter','latterly','least','less','ltd','made','many','may','me','meanwhile','might','mill','mine','more','moreover','most','mostly','move','much','must','my','myse\"','name','namely','neither','never','nevertheless','next','nine','no','nobody','none','noone','nor','not','nothing','now','nowhere','of','off','often','on','once','one','only','onto','or','other','others','otherwise','our','ours','ourselves','out','over','own','part','per','perhaps','please','put','rather','re','same','see','seem','seemed','seeming','seems','serious','several','she','should','show','side','since','sincere','six','sixty','so','some','somehow','someone','something','sometime','sometimes','somewhere','still','such','system','take','ten','than','that','the','their','them','themselves','then','thence','there','thereafter','thereby','therefore','therein','thereupon','these','they','thick','thin','third','this','those','though','three','through','throughout','thru','thus','to','together','too','top','toward','towards','twelve','twenty','two','un','under','until','up','upon','us','very','via','was','we','well','were','what','whatever','when','whence','whenever','where','whereafter','whereas','whereby','wherein','whereupon','wherever','whether','which','while','whither','who','whoever','whole','whom','whose','why','will','with','within','without','would','yet','you','your','yours','yourself','yourselves']
for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        print(e)
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)

    for word in words:
        word = word.lower()
        if word not in stop_words:
            key = word + "," + article_id
            print "%s\t%d" % (key, 1)

Overwriting mapper.py


In [95]:
%%writefile reducer.py
import sys
current_article_id = None
current_word = None
word_sum = 0
for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        word, article_id = key.split(',', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_article_id != article_id or current_word != word:
        if current_article_id and current_word:
            key = current_word + "," + current_article_id
            print "%s\t%d" % (key, word_sum)
            word_sum = 0
        current_article_id = article_id
        current_word = word
    word_sum += count
key = current_word + "," + current_article_id
print "%s\t%d" % (key, word_sum)

Overwriting reducer.py


In [96]:
%%bash
NUM_REDUCERS=8
OUT_DIR="tfidf_result_"$(date +"%s%6N")
hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Tfidf" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

rm: `tfidf_result_1522352174954688': No such file or directory
18/03/29 19:36:18 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/03/29 19:36:18 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/03/29 19:36:18 INFO mapred.FileInputFormat: Total input files to process : 1
18/03/29 19:36:18 INFO mapreduce.JobSubmitter: number of splits:2
18/03/29 19:36:18 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1522320992381_0004
18/03/29 19:36:19 INFO impl.YarnClientImpl: Submitted application application_1522320992381_0004
18/03/29 19:36:19 INFO mapreduce.Job: The url to track the job: http://d801e48a6a61:8088/proxy/application_1522320992381_0004/
18/03/29 19:36:19 INFO mapreduce.Job: Running job: job_1522320992381_0004
18/03/29 19:36:25 INFO mapreduce.Job: Job job_1522320992381_0004 running in uber mode : false
18/03/29 19:36:25 INFO mapreduce.Job:  map 0% reduce 0%
18/03/29 19:36:41 INFO mapreduce.Job:  map 6% reduce 0%
18/03/29 19:36:47 

In [97]:
! hdfs dfs -ls tfidf_result_1522352174954688/

Found 9 items
-rw-r--r--   1 jovyan supergroup          0 2018-03-29 19:39 tfidf_result_1522352174954688/_SUCCESS
-rw-r--r--   1 jovyan supergroup    6566868 2018-03-29 19:39 tfidf_result_1522352174954688/part-00000
-rw-r--r--   1 jovyan supergroup    6569591 2018-03-29 19:39 tfidf_result_1522352174954688/part-00001
-rw-r--r--   1 jovyan supergroup    6558042 2018-03-29 19:39 tfidf_result_1522352174954688/part-00002
-rw-r--r--   1 jovyan supergroup    6562065 2018-03-29 19:39 tfidf_result_1522352174954688/part-00003
-rw-r--r--   1 jovyan supergroup    6565405 2018-03-29 19:39 tfidf_result_1522352174954688/part-00004
-rw-r--r--   1 jovyan supergroup    6561957 2018-03-29 19:39 tfidf_result_1522352174954688/part-00005
-rw-r--r--   1 jovyan supergroup    6566523 2018-03-29 19:39 tfidf_result_1522352174954688/part-00006
-rw-r--r--   1 jovyan supergroup    6566207 2018-03-29 19:39 tfidf_result_1522352174954688/part-00007


In [100]:
! hdfs dfs -cat tfidf_result_1522352174954688/part-00000|head

0%however,6970	1
0&\mbox,3170	1
0(8)320-1231,2061	1
0)$and,4052	1
0,03,5188	1
0,1,0',3,2961	1
0,1,4664	10
0,1,5225	1
0,1,6042	1
0,1,\dots,n,4594	1
cat: Unable to write to output stream.


In [101]:
! hdfs dfs -cat tfidf_result_1522352174954688/part-00000| grep "^labor," | wc -l

31


In [102]:
! hdfs dfs -cat tfidf_result_1522352174954688/part-00001| grep "^labor," | wc -l

38


In [103]:
! hdfs dfs -cat tfidf_result_1522352174954688/part-00002| grep "^labor," | wc -l

42


In [104]:
! hdfs dfs -cat tfidf_result_1522352174954688/part-00003| grep "^labor," | wc -l

35


In [105]:
! hdfs dfs -cat tfidf_result_1522352174954688/part-00004| grep "^labor," | wc -l

33


In [106]:
! hdfs dfs -cat tfidf_result_1522352174954688/part-00005| grep "^labor," | wc -l

33


In [107]:
! hdfs dfs -cat tfidf_result_1522352174954688/part-00006| grep "^labor," | wc -l

32


In [108]:
! hdfs dfs -cat tfidf_result_1522352174954688/part-00007| grep "^labor," | wc -l

30


In [ ]:
#274 totally

In [109]:
! hdfs dfs -cat /data/wiki/en_articles_part/articles-part | wc -l

4100


In [112]:
! hdfs dfs -cat tfidf_result_1522352174954688/part-00006| grep "^labor,.*12$" | head

labor,12	12
labor,2597	12


In [ ]:
%%writefile mapper2.py
import sys
for line in sys.stdin:
    try:
        key, word_sum = line.split('\t', 1)
        word, article_id = key.split(',', 1)
    except ValueError as e:
        continue
    value = word + "," + word_sum
    print '%s\t%s' % (article_id, value)

In [ ]:
%%writefile reducer2.py
import sys
article_word_count = 0
current_article_id = None
article_words = {}
article_word_counts = {}
for line in sys.stdin:
    try:
        article_id, value = line.strip().split('\t', 1)
        word, word_sum = value.split(',', 1)
    except ValueError as e:
        continue

    if current_article_id != article_id:
        if current_article_id:
            article_word_counts[current_article_id] = article_word_count
            article_word_count = 0
        current_article_id = article_id
    try:
        article_word_count += int(word_sum)
        if article_id in article_words.keys():
            temp_list = article_words[article_id]
            temp_list.append(word + "\t" + word_sum)
            article_words[article_id] = temp_list
        else:
            temp_list = []
            temp_list.append(word + "\t" + word_sum) 
            article_words[article_id] = temp_list
    except:
        continue
article_word_counts[current_article_id] = article_word_count
for key in article_words.keys():
    items = article_words[key]
    for item in items:
        word, word_sum = item.strip().split("\t", 1)
        _key = word + "," + key
        _value = word_sum + "," + str(article_word_counts[key])
        print '%s\t%s' % (_key, _value)

In [47]:
%%bash
NUM_REDUCERS=8
OUT_INTERIM_DIR="tfidf_interim_result_"$(date +"%s%6N")
hdfs dfs -rm -r -skipTrash ${OUT_INTERIM_DIR} > /dev/null
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Tfidf" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper2.py,reducer2.py \
    -mapper "python mapper2.py" \
    -reducer "python reducer2.py" \
    -input tfidf_result_1522352174954688 \
    -output ${OUT_INTERIM_DIR} > /dev/null

rm: `tfidf_interim_result_1522347067568010': No such file or directory
18/03/29 18:11:10 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/03/29 18:11:11 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/03/29 18:11:11 INFO mapred.FileInputFormat: Total input files to process : 8
18/03/29 18:11:11 INFO mapreduce.JobSubmitter: number of splits:8
18/03/29 18:11:11 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1522320992381_0002
18/03/29 18:11:11 INFO impl.YarnClientImpl: Submitted application application_1522320992381_0002
18/03/29 18:11:11 INFO mapreduce.Job: The url to track the job: http://d801e48a6a61:8088/proxy/application_1522320992381_0002/
18/03/29 18:11:11 INFO mapreduce.Job: Running job: job_1522320992381_0002
18/03/29 18:11:17 INFO mapreduce.Job: Job job_1522320992381_0002 running in uber mode : false
18/03/29 18:11:17 INFO mapreduce.Job:  map 0% reduce 0%
18/03/29 18:11:24 INFO mapreduce.Job:  map 50% reduce 0%
18/03/29 

In [48]:
! hdfs dfs -ls tfidf_interim_result_1522347067568010/

Found 9 items
-rw-r--r--   1 jovyan supergroup          0 2018-03-29 18:11 tfidf_interim_result_1522347067568010/_SUCCESS
-rw-r--r--   1 jovyan supergroup    8634905 2018-03-29 18:11 tfidf_interim_result_1522347067568010/part-00000
-rw-r--r--   1 jovyan supergroup    8125858 2018-03-29 18:11 tfidf_interim_result_1522347067568010/part-00001
-rw-r--r--   1 jovyan supergroup    8501952 2018-03-29 18:11 tfidf_interim_result_1522347067568010/part-00002
-rw-r--r--   1 jovyan supergroup    8561766 2018-03-29 18:11 tfidf_interim_result_1522347067568010/part-00003
-rw-r--r--   1 jovyan supergroup    8613223 2018-03-29 18:11 tfidf_interim_result_1522347067568010/part-00004
-rw-r--r--   1 jovyan supergroup    8439894 2018-03-29 18:11 tfidf_interim_result_1522347067568010/part-00005
-rw-r--r--   1 jovyan supergroup    9553554 2018-03-29 18:11 tfidf_interim_result_1522347067568010/part-00006
-rw-r--r--   1 jovyan supergroup    8864716 2018-03-29 18:11 tfidf_interim_result_1522347067568010/part-0000

In [57]:
! hdfs dfs -cat tfidf_interim_result_1522347067568010/part-00002| grep "^labor,12" | head

labor,12	12,6092


In [75]:
! hdfs dfs -cat tfidf_interim_result_1522347067568010/part-00001| grep "^labor,1254" | head

labor,1254	2,2906


In [58]:
! hdfs dfs -cat tfidf_interim_result_1522347067568010/part-00002| grep "^labor,.*12,6092$" | head

labor,12	12,6092


In [59]:
%%writefile mapper3.py
import sys
for line in sys.stdin:
    key, value = line.strip().split('\t', 1)
    word, article_id = key.split(',', 1)
    word_sum, words_in_article = value.split(',', 1)
    value = article_id + "," + word_sum + "," + words_in_article + "," + str(1)
    print '%s\t%s' % (word, value)

Writing mapper3.py


In [60]:
%%writefile reducer3.py
import sys
import math
current_word = None
word_in_articles_count = 0
word_stats_dict = {}
word_in_articles_counts = {}
for line in sys.stdin:
    try:
      word, value = line.strip().split('\t', 1)
      word_stats = value.split(',')
      count = int(word_stats[3])
    except ValueError as e:
      continue

    if current_word != word:
        if current_word:
            word_in_articles_counts[current_word] = word_in_articles_count
            word_in_articles_count = 0
        current_word = word
    word_in_articles_count += int(count)
    
    if word in word_stats_dict.keys():
        word_stats_dict[word].append(word_stats[0] + "," + word_stats[1] + "," + word_stats[2])
    else:
        word_stats_dict[word] = []
        word_stats_dict[word].append(word_stats[0] + "," + word_stats[1] + "," + word_stats[2])
word_in_articles_counts[current_word] = word_in_articles_count
for word in word_in_articles_counts.keys():
    word_stats_list = word_stats_dict[word]
    for word_stats in word_stats_list:
        word_stats = word_stats.split(",")
        try :
          key = word + "," + word_stats[0]
          print "%s\t%s\t%s\t%s" % (key, word_stats[1], word_stats[2], word_in_articles_counts[word])
        except ZeroDivisionError as e:
          continue

Writing reducer3.py


In [61]:
%%bash
NUM_REDUCERS=8
OUT_FINAL_DIR="tfidf_final_result_"$(date +"%s%6N")
hdfs dfs -rm -r -skipTrash ${OUT_FINAL_DIR} > /dev/null
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Tfidf" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper3.py,reducer3.py \
    -mapper "python mapper3.py" \
    -reducer "python reducer3.py" \
    -input tfidf_interim_result_1522347067568010 \
    -output ${OUT_FINAL_DIR} > /dev/null

rm: `tfidf_final_result_1522348436765036': No such file or directory
18/03/29 18:34:00 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/03/29 18:34:00 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/03/29 18:34:00 INFO mapred.FileInputFormat: Total input files to process : 8
18/03/29 18:34:00 INFO mapreduce.JobSubmitter: number of splits:8
18/03/29 18:34:01 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1522320992381_0003
18/03/29 18:34:01 INFO impl.YarnClientImpl: Submitted application application_1522320992381_0003
18/03/29 18:34:01 INFO mapreduce.Job: The url to track the job: http://d801e48a6a61:8088/proxy/application_1522320992381_0003/
18/03/29 18:34:01 INFO mapreduce.Job: Running job: job_1522320992381_0003
18/03/29 18:34:06 INFO mapreduce.Job: Job job_1522320992381_0003 running in uber mode : false
18/03/29 18:34:06 INFO mapreduce.Job:  map 0% reduce 0%
18/03/29 18:34:13 INFO mapreduce.Job:  map 75% reduce 0%
18/03/29 18

In [62]:
! hdfs dfs -ls tfidf_final_result_1522348436765036/

Found 9 items
-rw-r--r--   1 jovyan supergroup          0 2018-03-29 18:54 tfidf_final_result_1522348436765036/_SUCCESS
-rw-r--r--   1 jovyan supergroup   10549246 2018-03-29 18:44 tfidf_final_result_1522348436765036/part-00000
-rw-r--r--   1 jovyan supergroup   10160181 2018-03-29 18:44 tfidf_final_result_1522348436765036/part-00001
-rw-r--r--   1 jovyan supergroup   10343749 2018-03-29 18:44 tfidf_final_result_1522348436765036/part-00002
-rw-r--r--   1 jovyan supergroup   10203016 2018-03-29 18:44 tfidf_final_result_1522348436765036/part-00003
-rw-r--r--   1 jovyan supergroup    9883640 2018-03-29 18:44 tfidf_final_result_1522348436765036/part-00004
-rw-r--r--   1 jovyan supergroup   10389818 2018-03-29 18:45 tfidf_final_result_1522348436765036/part-00005
-rw-r--r--   1 jovyan supergroup   10316351 2018-03-29 18:54 tfidf_final_result_1522348436765036/part-00006
-rw-r--r--   1 jovyan supergroup    9760845 2018-03-29 18:53 tfidf_final_result_1522348436765036/part-00007


In [70]:
! hdfs dfs -cat tfidf_final_result_1522348436765036/part-00007| grep "^labor,12" | head

labor,12	12	6092	274
labor,1254	2	2906	274


In [72]:
! hdfs dfs -cat tfidf_final_result_1522348436765036/part-00007| grep "^labor,12.*274$" | head

labor,12	12	6092	274
labor,1254	2	2906	274


In [88]:
# float(12/(126092*math.log10(1+275))) = 3.898900116685244e-05
# (12/6092) * (1/math.log(1+274)) = 0.00035069907961617183
# should be 0.000350696420907907
# 274 should be 275 ?
# 3.898900116685244e-05
# 4100 articles

In [ ]:
%%bash
NUM_REDUCERS=8

OUT_DIR="tfidf_result_"$(date +"%s%6N")
hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Tfidf" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null
    
OUT_INTERIM_DIR="tfidf_interim_result_"$(date +"%s%6N")
hdfs dfs -rm -r -skipTrash ${OUT_INTERIM_DIR} > /dev/null
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Tfidf" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper2.py,reducer2.py \
    -mapper "python mapper2.py" \
    -reducer "python reducer2.py" \
    -input ${OUT_DIR} \
    -output ${OUT_INTERIM_DIR} > /dev/null
    
OUT_FINAL_DIR="tfidf_final_result_"$(date +"%s%6N")
hdfs dfs -rm -r -skipTrash ${OUT_FINAL_DIR} > /dev/null
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Tfidf" \
    -D mapreduce.job.reduces=8 \
    -files mapper3.py,reducer3.py \
    -mapper "python mapper3.py" \
    -reducer "python reducer3.py" \
    -input ${OUT_INTERIM_DIR} \
    -output ${OUT_FINAL_DIR} > /dev/null 2>${OUT_FINAL_DIR}

cat ${OUT_FINAL_DIR} >&2
grep 'labor,12' ${OUT_LOG} \
    | cut -d'=' -f2 \
    | paste -sd' ' \
    | awk '{ printf "%.2f", ($1 / $2 * 100) }'